In [2]:
import torch
from torch.nn import MSELoss

In [5]:
t = torch.rand((1, 40, 18))

In [7]:
t = t.squeeze()
t.shape

torch.Size([40, 18])

In [10]:
a1 = t[:, [0,1]]
a2 = t[:, [6,7]]
b  = t[:, [12,13]]

In [11]:
a1.shape, a2.shape, b.shape

(torch.Size([40, 2]), torch.Size([40, 2]), torch.Size([40, 2]))

In [13]:
from torch.nn import PairwiseDistance
dist = PairwiseDistance(p=2)

In [18]:
dis_a1_a2 = dist(a1, a2)
dis_a1_b = dist(a1, b)
dis_b_a2 = dist(b, a2)
dis_a1_a2.shape

torch.Size([40])

In [21]:
distances = torch.stack([dis_a1_a2, dis_a1_b, dis_b_a2], dim = 1)
distances.shape

torch.Size([40, 3])

In [24]:
dis = distances.unsqueeze(dim=0)
dis.shape

torch.Size([1, 40, 3])

In [26]:
out = torch.rand((1,40,18))
input = torch.cat([out, dis], dim=2)
input.shape

torch.Size([1, 40, 21])

In [27]:
for j in range(5):
    print(j)

0
1
2
3
4


In [38]:
def get_data_by_type(out, label, i):
    os, ls = [], []
    
    for j in range(3):
        o = out[:, 2*i + (6*j) : 2*(i+1) + (6*j)]
        os.append(o)
        l = label[:, 2*i + (6*j) : 2*(i+1) + (6*j)]
        ls.append(l)
    
    out = torch.cat(os, dim=1)
    label = torch.cat(ls, dim=1)
    
    return out, label

In [39]:
t = torch.rand((4, 18))
l = torch.rand((4, 18))
o_losses = []
t_losses = []

In [40]:
lf = MSELoss()

In [41]:
for i in range(3):
    # Object specific loss
    out = t[:, i*6 : (i+1)*6]
    label = l[:, i*6 : (i+1)*6]
    object_loss = lf(out, label)
    o_losses.append(object_loss.item())
    
    # Data specific loss (coords, orientation, force)
    out2, label2 = get_data_by_type(t, l, i)
    data_loss = lf(out2, label2)
    t_losses.append(data_loss.item())

In [42]:
o_losses, t_losses

([0.22520773112773895, 0.12353353947401047, 0.1602531373500824],
 [0.09188965708017349, 0.22642183303833008, 0.19068290293216705])

In [43]:
sum(o_losses), sum(t_losses)

(0.5089944079518318, 0.5089943930506706)